In [12]:
# Libraries here
import pandas as pd
import numpy as np
import requests
import json

!pip install edist
import edist.ted as ted

Defaulting to user installation because normal site-packages is not writeable


In [13]:
insertion_cost = 1.
deletion_cost = 1.
leave_change = 1. 
default_cost = 100

In [14]:
# API call to iSeeOntoAPI to get the most similar cases

url = "https://api-dev.isee4xai.com/api/trees/cbr_retrieve"

payload = json.dumps({
  "treeId": "64b676baa737e466ce27f166",
  "usecaseId": "6426a68da3402ba28c44a7c0",
  "topk": 5
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

[{"name":"Tree","score__":7,"usecase":"6426a68da3402ba28c44a7c0","persona":"6426a7eca3402ba28c44a7d0","intent":"intent-130431","description":"","path":"b3projects-b22b93e4-0449-41fa-8876-48551541154a","data":{"version":"0.1.0","scope":"project","selectedTree":"33def3ec-31a8-47c1-856c-7fd724718df2","trees":[{"version":"0.1.0","scope":"tree","id":"33def3ec-31a8-47c1-856c-7fd724718df2","Instance":"Explanation Experience","description":"","root":"546f5cee-68b0-4b90-85be-786b9957d03a","properties":{},"nodes":{"5112868d-f790-4665-ab3e-18a36a857363":{"id":"5112868d-f790-4665-ab3e-18a36a857363","Concept":"Sequence","Instance":"Sequence","description":"","properties":{},"display":{"x":-60,"y":168},"firstChild":{"Id":"85b9b22e-1b0a-4a9b-81a9-83952d27271a","Next":{"Id":"5829d6db-5011-4ad8-846a-ab8452c6be46","Next":null}}},"546f5cee-68b0-4b90-85be-786b9957d03a":{"id":"546f5cee-68b0-4b90-85be-786b9957d03a","Concept":"Priority","Instance":"Priority","description":"","properties":{},"display":{"x":-6

In [15]:
# Build the node list from the behavior tree for the graph structure
def print_node_instances(node_id, nodes_dict):
    node = nodes_dict[node_id]
    node_instance = node['Instance']
    if node_instance is None:
        return None
    node_list.append(node_instance)
    id_list.append(node_id)

    if 'firstChild' in node:
        first_child_id = node['firstChild']['Id']
        print_node_instances(first_child_id, nodes_dict)
        next_child = node['firstChild'].get('Next')

        while next_child is not None:
            next_child_id = next_child['Id']
            print_node_instances(next_child_id, nodes_dict)
            next_child = next_child.get('Next')

    return node_list, id_list

In [16]:
# Get the index of the parent in the behavior tree
def get_index(node_id, nodes_dict, id_list):
    node = nodes_dict[node_id]
    node_instance = node.get('Instance')
    node_index = id_list.index(node_id)
    node_index = node_index + 1 #node list start with 'r'

    return node_index, node_instance

In [17]:
# Find the parent of a node
def find_parent(node_id, node, parent_child_dict, id_list, nodes_dict):
    parent_index, parent_instance = get_index(node_id, nodes_dict, id_list)

    if 'firstChild' in node:
        first_child_id = node['firstChild']['Id']
        child_index, child_instance = get_index(first_child_id, nodes_dict, id_list)

        if parent_index not in parent_child_dict:
            parent_child_dict[parent_index] = []
        if child_index not in parent_child_dict[parent_index]:
            parent_child_dict[parent_index].append(child_index)

        next_child = node['firstChild'].get('Next')
        while next_child is not None:
            next_child_id = next_child['Id']
            child_index, child_instance = get_index(next_child_id, nodes_dict, id_list)
            if child_index not in parent_child_dict[parent_index]:
                parent_child_dict[parent_index].append(child_index)  # Add child index to the parent's list
            next_child = next_child.get('Next')

        return parent_instance

In [18]:
# Build a parent-child dictionary for nodes in the bahavior tree
def create_parent_child_dict(nodes_dict, node_list, id_list):
    parent_child_dict = {}
    # root = node_list[0] #r
    parent_child_dict[0] = [1]  # Add root node with index 0

    for i, (instance, node_id) in enumerate(zip(node_list[1:], id_list), start=1):
        node_index = i
        node_id =id_list[node_index-1]
        node = nodes_dict[node_id]
        find_parent(node_id, node, parent_child_dict, id_list, nodes_dict)

    return parent_child_dict

In [23]:
# function to translate the case solution to graph structure
# This function must work for all the cases and the query
# TODO
def translateCasesFromJSONtoGraph(case):
    ## Preeja can provide this
    tree_dict, parent_child_dict = {},{}
    # node_list = ['r'] # Added 'r' as the default root node in the node list
    # id_list =[] #List of node id's

    for idx, obj in enumerate(case, start=1):
        trees = obj['data']['trees']
        # Get the 'nodes' from 'trees'
        for tree in trees:
            nodes_dict = {}
            nodes = tree.get('nodes', {})
            nodes_dict.update(nodes)
            # Get the root node
            root_node_id = tree.get('root')

        # Call the recursive function to print node instances
        node_list, id_list= print_node_instances(root_node_id, nodes_dict, node_list=['r'], id_list=[])
        # Call the function to create the parent_child dictionary
        parent_child_dict = create_parent_child_dict(nodes_dict, node_list, id_list)
        # Build the adjacency list from the behavior tree
        adjacency_list = build_adjacency_list(node_list, parent_child_dict)

        tree_key = f'tree_{idx}'
        tree_dict[tree_key] = {
              'tree_json': trees,
              'tree_graph': {
                  'nodes': node_list,
                  'adj': adjacency_list
              }
        }

    # # Print the dictionary
    # print('\n',tree_dict)

    return tree_dict

In [24]:
## Preeja can provide this
# Load case base from json file
with open("apioutput.json", "r") as f:
    case = json.load(f)
tree_dict = translateCasesFromJSONtoGraph(case)

TypeError: print_node_instances() got an unexpected keyword argument 'node_list'

In [ ]:
print(tree_dict)

# New section

In [ ]:
# delta: custom node distance function
def semantic_delta(x, y):
    df = getSimilarityTable()
    #print(df["/Images/Anchors"]["/Images/Counterfactuals"])

    if(x==y):
        ret = 0.
    elif(x!=None and y==None): #inserting
        #print("inserting")
        ret = insertion_cost
    elif(x==None and y!=None): #deleting
        #print("deleting")
        ret = deletion_cost
    elif(x=='r'or y=='r'):  #we assign an infinite cost when comparing a root node
        #print("root")
        ret = np.inf
    elif(x in ['f','t'] and y in ['f','t']): #if both nodes are either failer or succeeder, assign null cost
        #print("failer and succeeder")
        ret = 0.
    elif(x in ['s','p'] and y in['s','p']): #if both nodes are either sequence or priority, assign null cost
        #print("sequence and priority")
        ret = 0.
    elif(x in ['s','p'] or y in ['s','p']): #if one of the nodes is a sequence or priority, the other won't because of the previous rule
        #print("sequence or priority")
        ret = np.inf
    elif(x in ['f','t'] and y[0]=='/'):
        #print("cambiando explainer por failure or succeeder")
        ret = leave_change
    elif(x[0]=='/' and y in ['f','t']):
        #print("cambiando explainer por failure or succeeder")
        ret = leave_change
    elif x in df.columns and y in df.columns: #Both explainers are in similarity table, DF MUST BE LOADED BEFOREHAND
        ret = 1-df.loc[x][y]
    else: # here we have a question leave
        ret = 10 #default_cost  
       
    #print('sem_delta: ',str(x)," , "+str(y)+ " = "+ str(ret) )   
    return ret

In [ ]:
# Function to calculate the similarity between two BTs, both of them have to have graph structure
def editDistFunc(x_nodes, x_adj, y_nodes, y_adj, delta):
    # delta=custom node distance function
    if delta is None:
        return ted.standard_ted(x_nodes, x_adj, y_nodes, y_adj)
    else:
        return ted.ted(x_nodes, x_adj, y_nodes, y_adj, delta)

In [ ]:
# Next, we have to get each case BT from response.text
# save this in a structure (list, dictionary...)
# TODO

In [ ]:
# Select the subtree from the query that we have to consider, given a specific condition node
# TODO

In [ ]:
# Translate each BT (json format) to graph structure (list of nodes and adjacency list)
# Also, we should use another structure to save the correspondence between the json format
# and the graph format, so later, when we have to return the json format for the most similar BT
# we ca access that structure and not doing the translation again
# TODO
# call to translateCases(case)

In [ ]:
# Adapt the similarity metric function between explainers
# Think of how to do it
# TODO


In [ ]:
# MAIN
# solution = structure
# for every BT in the case base:
#   compare the query with that BT
#   solution[BT] = editDistFunc(x_nodes, x_adj, y_nodes, y_adj, delta)

# Sort solution to get the BT with the lowest edit distance

# From the structure above, we have to get the json format for that solution
